In [24]:
# Importa biblioteca
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .master("yarn")
        .appName("Data Transform")
        .getOrCreate()
)

# Importar tabela a partir de um banco postgresql
df = spark.read.format("jdbc") \
	.option("url","jdbc:postgresql://10.13.123.0:5432/curso_spark") \
	.option("dbtable","processos") \
	.option("user","postgres") \
	.option("password","spark") \
	.option("driver","org.postgresql.Driver") \
	.load()

In [25]:
df.count()

300673

In [27]:
df_clean = df.drop_duplicates()
df_clean.count()

164099

In [11]:
from pyspark.sql.functions import desc, sum

In [29]:
df_clean.show()

[Stage 58:>                                                         (0 + 1) / 1]

+-------+----+------+------------+------------+--------+------+------------+---------------+----------+--------------------+
|ID_PROC|VARA|CLASSE|QTD_RDA_PROC|QTD_RTE_PROC|ID_FAIXA|ID_RDA|QTD_ASSUNTOS|    FAIXA_CAUSA|TEMPO_DIAS|       ARRAY_ASSUNTO|
+-------+----+------+------------+------------+--------+------+------------+---------------+----------+--------------------+
|  42975|1572|    58|           2|           1|       4|  4767|           9|  30.001-40.000|      2393|"Adicional Noturn...|
| 153687|1525|    77|           1|           1|      47| 13640|           6|460.001-470.000|        56|"Anotação / Baixa...|
|  73172|1502|    42|           1|           2|       2| 24444|           2|  10.001-20.000|      1156|"Multa de 40% do ...|
|  50223|1489|    77|           2|           1|       4| 80013|          15|  30.001-40.000|      2430|"Cálculo / Reperc...|
|  34958|1487|    77|           6|           1|       4|  3141|           1|  30.001-40.000|      2660|Multa do Artigo 4...|


In [40]:
# agrupando total de reus por vara
df_agrupado_vara = df_clean.groupBy('VARA').agg(sum('QTD_RDA_PROC').alias('TOTAL_REUS'), sum('QTD_RTE_PROC').alias('TOTAL_RECLAMANTES')).orderBy(desc('TOTAL_REUS'))

In [41]:
df_agrupado_vara.show()

[Stage 79:>                                                         (0 + 1) / 1]

+----+----------+-----------------+
|VARA|TOTAL_REUS|TOTAL_RECLAMANTES|
+----+----------+-----------------+
|1514|      7134|             4578|
|1529|      5039|             3091|
|1528|      4856|             3015|
|1520|      4717|             3308|
|1516|      4467|             2785|
|1518|      4299|             2808|
|1515|      4285|             2372|
|1504|      4190|             2405|
|1503|      4147|             2350|
|1511|      4119|             2346|
|1505|      4110|             2309|
|1506|      4034|             2817|
|1477|      3957|             2192|
|1480|      3943|             2418|
|1494|      3925|             2203|
|1519|      3881|             2451|
|1500|      3872|             2194|
|1492|      3849|             2273|
|1478|      3823|             2274|
|1474|      3758|             2091|
+----+----------+-----------------+
only showing top 20 rows



In [43]:
# Gravar dados no banco postgresql
df_agrupado_vara.write.format("jdbc") \
	.option("url","jdbc:postgresql://10.13.123.0:5432/curso_spark") \
	.option("dbtable","carlos") \
	.option("user","postgres") \
	.option("password","spark") \
	.option("driver","org.postgresql.Driver") \
	.save()

In [44]:
# Importar tabela a partir de um banco postgresql
df = spark.read.format("jdbc") \
	.option("url","jdbc:postgresql://10.13.123.0:5432/curso_spark") \
	.option("dbtable","carlos") \
	.option("user","postgres") \
	.option("password","spark") \
	.option("driver","org.postgresql.Driver") \
	.load()

df.show()

+----+----------+-----------------+
|VARA|TOTAL_REUS|TOTAL_RECLAMANTES|
+----+----------+-----------------+
|1514|      7134|             4578|
|1529|      5039|             3091|
|1528|      4856|             3015|
|1520|      4717|             3308|
|1516|      4467|             2785|
|1518|      4299|             2808|
|1515|      4285|             2372|
|1504|      4190|             2405|
|1503|      4147|             2350|
|1511|      4119|             2346|
|1505|      4110|             2309|
|1506|      4034|             2817|
|1477|      3957|             2192|
|1480|      3943|             2418|
|1494|      3925|             2203|
|1519|      3881|             2451|
|1500|      3872|             2194|
|1492|      3849|             2273|
|1478|      3823|             2274|
|1474|      3758|             2091|
+----+----------+-----------------+
only showing top 20 rows

